In [1]:
from typing import Optional, List, Union
import cv2
import numpy as np
import pytorch_lightning as pl
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import MNIST
from pytorch_lightning.loggers import WandbLogger

This is image classification with few good practices.

Uses the following frameworks:

- PyTorch Lightning
- TorchMetrics
-

In [2]:
class MnistDataset(pl.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        data_path: str,
        val_split: float,
        dataloader_num_workers: int = 0,
        **_,
    ):
        super().__init__()

        self.h = 28
        self.w = 28
        self.dims = (1, self.h, self.w)

        self.data_path = data_path
        self.batch_size = batch_size
        self.val_split = val_split
        self.num_workers = dataloader_num_workers

        self.transform_train_val, self.transform_test = self._get_transforms()

        self.mnist_train, self.mnist_val, self.mnist_test = None, None, None

    def prepare_data(self):
        # download
        MNIST(self.data_path, train=True, download=True)
        MNIST(self.data_path, train=False, download=True)

    def setup(self, stage: Optional[str] = None):
        """
        :param stage: One of {fit, validate, test}. None = all 3
        :return:
        """

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(
                self.data_path, train=True, transform=self.transform_train_val
            )
            full_size = len(mnist_full)
            val_size = int(full_size * self.val_split)
            self.mnist_train, self.mnist_val = random_split(
                mnist_full,
                [full_size - val_size, val_size],
                generator=torch.Generator().manual_seed(42),
            )

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(
                self.data_path, train=False, transform=self.transform_test
            )

    def train_dataloader(self):
        return DataLoader(
            self.mnist_train,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.mnist_val,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.mnist_test,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def _get_transforms(self):
        resize_transform = A.Compose(
            [
                A.PadIfNeeded(
                    min_height=self.h, min_width=self.w, border_mode=cv2.BORDER_CONSTANT
                ),
                A.RandomCrop(self.h, self.w),
            ]
        )
        augmentations = A.Compose(
            [
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.2),
            ]
        )

        def _normalizer(image, **_):
            return image / 255.0

        compatible = A.Compose(
            [
                ToTensorV2(always_apply=True),
                A.Lambda(image=_normalizer),
            ]
        )

        transforms_train_val_ = A.Compose([resize_transform, augmentations, compatible])
        transforms_test_ = A.Compose([resize_transform, compatible])

        def transforms_train_val(image):
            return transforms_train_val_(image=np.array(image))["image"]

        def transforms_test(image):
            return transforms_test_(image=np.array(image))["image"]

        return transforms_train_val, transforms_test

In [3]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))


class SimpleCNN(nn.Module):
    def __init__(
        self,
        architecture: List[Union[tuple, str, list]],
        in_channels: int,
    ):
        super(SimpleCNN, self).__init__()
        layers = []
        for module in architecture:
            if type(module) is tuple:
                layers.append(self._get_cnn_block(module, in_channels))
                in_channels = module[1]
            elif module == "M":
                layers.append(
                    nn.MaxPool2d(
                        kernel_size=(2, 2),
                        stride=(2, 2),
                    )
                )
            elif type(module) is list:
                for i in range(module[-1]):
                    for j in range(len(module) - 1):
                        layers.append(self._get_cnn_block(module[j], in_channels))
                        in_channels = module[j][1]
        self.model = nn.Sequential(*layers)

    @staticmethod
    def _get_cnn_block(module: tuple, in_channels):
        kernel_size, filters, stride, padding = module
        return CNNBlock(
            in_channels,
            filters,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
        )

    def forward(self, x):
        return self.model(x)

In [4]:
class MnistModel(pl.LightningModule):
    def __init__(self, **hp):
        super().__init__()
        architecture = [
            # (kernel_size, filters, stride, padding)
            (3, 10, 1, 1),
            "M",
            (3, 20, 1, 1),
            "M",
            (3, 40, 1, 1),
            # "M",
            (3, 80, 1, 1),
            (3, 10, 1, 0),
        ]

        self.model = nn.Sequential(
            SimpleCNN(architecture, in_channels=1),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
        )

        self.hp = hp
        self.save_hyperparameters(hp)

        self.criterion = nn.CrossEntropyLoss()

        # Metrics
        self.accuracy_train = pl.metrics.Accuracy()
        self.accuracy_val = pl.metrics.Accuracy()


    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        preds = self(images)
        loss = self.criterion(preds, targets)
        return {'loss': loss, 'preds': preds, 'inputs': images, 'targets': targets}

    def training_step_end(self, outs: dict):
        self.accuracy_train(outs["preds"], outs["targets"])
        self.log({"train/acc_step": self.accuracy_train})

    def validation_step(self, batch, batch_idx):
        return self._evaluate(batch, batch_idx, "val")

    def _evaluate(self, batch, batch_idx, prefix: str):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        self.log(f"{prefix}/loss", loss)
        return loss

In [5]:

def debug1():
    config = {
        "data_path": "../data",
        "val_split": 0.3,
        "batch_size": 2,
        "output_path": "./output",
        "model_save_frequency": 5,
        "dataloader_num_workers": 0,
    }
    dataset = MnistDataset(**config)

    dataset.prepare_data()
    dataset.setup()
    loader = dataset.train_dataloader()
    for image, classes in loader:
        print(image.shape, classes.shape)
        break

    hp = {
        "epochs": 10,
        "lr_initial": 0.0001,
        "lr_decay_every": 30,
        "lr_decay_by": 0.3,
    }
    model = MnistModel(**hp, **config)
    y = model(image)
    print(y.shape)

debug1()


torch.Size([2, 1, 28, 28]) torch.Size([2])
torch.Size([2, 10])


/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-jhk0fkj8/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /tmp/pip-req-build-jhk0fkj8/c10

In [ ]:

def train():

    hp = {
        "epochs": 10,
        "lr_initial": 0.0001,
        "lr_decay_every": 30,
        "lr_decay_by": 0.3,
    }

    config = {
        "data_path": "../data",
        "val_split": 0.3,
        "batch_size": 64,
        "output_path": "./output",
        "model_save_frequency": 5,
        "dataloader_num_workers": 0,
    }

    dataset = MnistDataset(**config)
    model = MnistModel(**hp, **config)
    wandb_logger = WandbLogger(project='classification_test', log_model=True)
    trainer = pl.Trainer(
        gpus=0,
        max_epochs=hp["epochs"],
        default_root_dir=config["output_path"],
        logger=wandb_logger,
    )
    # wandb_logger.watch(model)

    trainer.fit(model, datamodule=dataset)


train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
wandb: Currently logged in as: akhilez (use `wandb login --relogin` to force relogin)



  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 45.4 K
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
45.4 K    Trainable params
0         Non-trainable params
45.4 K    Total params
0.182     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [10]:
accuracy_train = pl.metrics.Accuracy()
accuracy_train(  # 0.5
    torch.tensor([[0.0, 0.1, 0.9], [0.0, 0.8, 0.2]]),
    torch.tensor([2, 2])
)
print(accuracy_train.compute())
accuracy_train(  # 1
    torch.tensor([[0.0, 0.1, 0.9], [0.0, 0.8, 0.2]]),
    torch.tensor([2, 1])
)
print(accuracy_train.compute())
accuracy_train(  # 3/4
    torch.tensor([[0.0, 0.1, 0.9], [0.0, 0.8, 0.2], [0.0, 0.1, 0.9], [0.0, 0.8, 0.2]]),
    torch.tensor([2, 0, 0, 0])
)
print(accuracy_train.compute())

tensor(0.5000)
tensor(0.7500)
tensor(0.5000)
